# MNIST classification with Qiskit

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.visualization import array_to_latex
from qiskit.quantum_info.operators import Operator, Pauli

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy.physics.quantum.qubit import matrix_to_qubit
from scipy.optimize import minimize
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
def sim_state(qc, disp=True):
    '''
    量子回路qcの量子状態を取得する。

    Args:
        qc (QuantumCircuit): 量子回路
        disp (Bool): Trueにするとketベクトルの線型結合の形で量子状態表記を得る。
    '''
    sim = Aer.get_backend('statevector_simulator')
    qc = transpile(qc, backend=sim)
    res = sim.run(qc).result()
    state = res.data()['statevector']
    if disp:
        ket = matrix_to_qubit(np.array(state)[:, np.newaxis])
        print(ket)
    return state

In [3]:
def mag_exp(qc, n):
    '''
    引数に与えられた期待値を計算する。

    Args:
        qc (QuantumCircuit): 量子回路
        n (int): qubit数

    Returns:
        y (float): 期待値
    '''
    state = sim_state(qc, disp=False)
    op = Operator(np.zeros([2**n, 2**n]))
    for k in range(n):
        op_str = ''
        for l in range(n):
            if l == k:
                op_str += 'Z'
            else:
                op_str += 'I'
        op += Operator(Pauli(op_str))
        
    y = state.expectation_value(op)/n
    return y

In [4]:
def U_in(x, n):
    '''
    Args:
        x データ。-1〜1に規格化された
    '''
    qc = QuantumCircuit()
    qr = QuantumRegister(n)
    qc.add_register(qr)

    angle = np.arcsin(x)
    qc.rx(angle, qr)
    
    U_in = qc.to_gate()
    U_in.name = 'U_in'

    return U_in

In [5]:
def U_in_for_MNIST(image_data, n, encoding_method=None):
    '''
    MNISTの画像データのインプット

    Args:    
        image_data (pandas.core.series.Series): 1枚のMNIST画層データ。shapeは(784, )
        n (int): qubit数
        encoding_method (str): エンコーディング方法の指定。
                                "amplitude", "basis", "angle"のいずれかを指定すること。
    '''
    qc = QuantumCircuit()
    qr = QuantumRegister(n)
    qc.add_register(qr)
    
    # データを正規化
    norm = np.linalg.norm(image_data)
    image_data = image_data / norm
    
    # 量子状態にエンコード(amplitude encoding)
    qc.initialize(image_data.tolist(), range(n))
    
    # U_in = qc.to_gate()
    # U_in.name = 'U_in'
    
    return qc

In [6]:
def QCLinput(x, n):
    '''
    量子回路を作るための関数

    Args:
        x (pandas.core.series.Series): 1枚のMNIST画層データ。shapeは(784, )
        n (int): qubit数
    '''
    qc = QuantumCircuit()
    qr = QuantumRegister(n)
    qc.add_register(qr)
    qc.append(U_in_for_MNIST(x, n), qr)    
    return qc

In [7]:
def U_rot(n, params):
    '''
    Args:
        n (int):
        params (list): rx: 0, 1, 2, ..., n-1
                       ry: n, n+1, n+2, ..., n+n-1
                       rz: 2n, 2n+1, 2n+2, ..., 2n+n-1
    '''
    qc = QuantumCircuit()
    qr = QuantumRegister(n)
    qc.add_register(qr)
    for k in range(n):
        qc.rx(params[k], qr[k])
        qc.ry(params[n+k], qr[k])
        qc.rz(params[2*n+k], qr[k])
    Urot = qc.to_gate()
    Urot.name = 'Urot'
    return Urot

In [8]:
def U_ent(n):
    '''
    entanglement
    '''
    qc = QuantumCircuit()
    qr = QuantumRegister(n)
    qc.add_register(qr)

    if n > 1:
        for k in range(n-1):
            qc.cz(qr[k], qr[k+1])
        qc.cz(qr[n-1], qr[0])

    Uent = qc.to_gate()
    Uent.name = 'U_ent'
    return Uent    

In [9]:
# MNISTデータの準備
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
# MNISTデータ確認
print(X.shape)  # (70000, 784)
print(y.shape)  # (70000,)
print(X_train.shape, X_test.shape) # (56000, 784) (14000, 784)
print(y_train.shape, y_test.shape) # (56000,) (14000,)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

display(X_train)
display(y_train)

(70000, 784)
(70000,)
(56000, 784) (14000, 784)
(56000,) (14000,)


,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0        5
1        4
2        8
3        0
4        2
        ..
55995    6
55996    6
55997    1
55998    0
55999    0
Name: class, Length: 56000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [ ]:
# TODO: データの正規化


In [11]:
# PCAで784次元から8次元に圧縮
pca = PCA(n_components=8)
pca.fit(X_train)
X_train_reduced = pd.DataFrame(pca.transform(X_train))
X_test_reduced = pd.DataFrame(pca.transform(X_test))

print(X_train_reduced.shape)
print(X_test_reduced.shape)
print(y_train.shape)
print(y_test.shape)

n_train = X_train_reduced.shape[0]
n_test = X_test_reduced.shape[0]

(56000, 8)
(14000, 8)
(56000,)
(14000,)


In [12]:
X_train_reduced.shape[0]

56000

In [13]:
# plt.plot(x_series, y_series)
# plt.show()

#### 学習

In [14]:
n = 3
depth = 3
params = np.random.rand(3*n*depth)*2*np.pi

In [15]:
type(y_train[0])

str

In [16]:
def cost_func_for_MNIST(params):
    '''
    コスト関数
    '''
    cost_total = 0
    for k in range(n_train):
        x = X_train_reduced.iloc[k, :]
        qc = QCLinput(x, n)
        qr = qc.qubits
        for d in range(depth):
            qc.append(U_ent(n), qr)
            qc.append(U_rot(n, params[d*3*n:(d+1)*3*n]), qr)
        y = mag_exp(qc, n)
        cost = 0.5*(int(y_train[k])-y)**2
        cost_total += cost
    cost_total /= n_train
    return cost_total

In [17]:
# 勾配を使わない、探索的な最適化方法
result =  minimize(cost_func_for_MNIST, params, method='COBYLA', options={'maxiter': 50})


KeyboardInterrupt



In [ ]:
# 0に近いほうが良い。
result.fun

In [ ]:
result.x

---
#### 推論

In [ ]:
# 学習済みのパラメータを使って推論
y_series = []
params = result.x

for k in range(n_test):
    x = X_test_reduced.iloc[k, :]
    qc = QCLinput(x, n)
    qr = qc.qubits
    for d in range(depth):
        qc.append(U_ent(n), qr)
        qc.append(U_rot(n, params[d*3*n:(d+1)*3*n]), qr)
    y = mag_exp(qc, n)
    y_series.append(y)

In [ ]:
print(y_series.shape)
print(y_test.shape)

In [ ]:
print(y_series)

In [ ]:
print(y_test)